In [1]:
import ipywidgets as widgets
from IPython.display import display

import yfinance as yf
import pandas as pd
import requests
import math
import time

import matplotlib.pyplot as plt

from datetime import datetime, timedelta

# Widget for the ticker symbol
ticker_input = widgets.Text(
    value='SPY',  # Default value
    description='Ticker:',
    disabled=False
)

# Widget for the start year 
start_year_input = widgets.Text(
    value='2022-01-01',  # Default value
    description='Start Date:',
    disabled=False
)

# Widget for the number of days
days_input = widgets.IntText(
    value=5,  # Default value
    description='Days:',
    disabled=False
)

display(ticker_input, start_year_input, days_input)

button = widgets.Button(description="Run Analysis")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    # Clear the previous output
    output.clear_output()
    
    # Your code to run when the button is clicked
    # Example: print the values from the input widgets
    with output:
        tickers = ticker_input.value.replace(' ','').split(',')
        tickers

        results = pd.DataFrame(columns=['Ticker','Moving Average','Extension','Count','Mean','Std','Min','25 Percentile','50 Percentile','75 Percentile','Maximum'])
        results

        for ticker in tickers:

            df = yf.download(ticker, start=start_year_input.value, end=datetime.now().strftime('%Y-%m-%d'))

            df['Daily 50 SMA'] = df['Close'].rolling(window=50).mean()
            df['Daily 200 SMA'] = df['Close'].rolling(window=200).mean()

            df['Daily 8 EMA'] = df['Close'].ewm(span=8, adjust=False).mean()
            df['Daily 21 EMA'] = df['Close'].ewm(span=21, adjust=False).mean()

            df['Open Distance 8 EMA'] = ((df['Open'] - df['Daily 8 EMA']) / df['Daily 8 EMA']) * 100
            df['Open Distance 21 EMA'] = ((df['Open'] - df['Daily 21 EMA']) / df['Daily 21 EMA']) * 100

            df['Open Distance 50 SMA'] = ((df['Open'] - df['Daily 50 SMA']) / df['Daily 50 SMA']) * 100
            df['Open Distance 200 SMA'] = ((df['Open'] - df['Daily 200 SMA']) / df['Daily 200 SMA']) * 100

            df.reset_index(inplace=True)

            for col in df.columns[-4:]:

                for distance in range(math.floor(df[col].min()), math.ceil(df[col].max())):
                    hold = df[(df[col] >= distance) & (df[col] <= distance+1)].reset_index(drop=True)

                    trades = [] 

                    for i in range(len(hold)):
                        try:
                            start_index_original = df[df['Date'] == hold.at[i,'Date']].index[0] # Denotes index position of day in original dataframe

                            price_open = df.iloc[start_index_original]['Open']
                            price_close = df.iloc[start_index_original+days_input.value]['Close']

                            trades.append(((price_close - price_open) / price_open) * 100)
                        except:
                            continue


                    results.loc[len(results)] = [ticker, col, (distance, distance+1)] + [c for c in pd.Series(trades, dtype='float64').describe().values]

        width = 20
        height = 25
        rotation = 70

        plt.figure(figsize=(width,height))

        plt.subplot(4,1,1)

        mean = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 8 EMA'].groupby('Extension')['Mean'].mean()).reset_index()
        count = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 8 EMA'].groupby('Extension')['Count'].sum()).reset_index()

        holder = pd.merge(count, mean, how='left', on='Extension')
        holder['Extension'] = holder['Extension'].astype(str)
        holder

        holder = holder[holder[holder['Count'] > 1].index[0]:holder[holder['Count'] > 1].index[-1]+1].reset_index(drop=True)

        # Creating the bar chart
        plt.bar(holder['Extension'], holder['Mean'])

        # Annotating the bars with 'Label' values
        for idx, row in holder.iterrows():
            plt.annotate(row['Count'], (row['Extension'], row['Mean']), textcoords="offset points", xytext=(0,20), ha='center')

        plt.xlabel('Extension from 8 EMA')
        plt.ylabel('Mean Return')
        plt.title('8 EMA')
        plt.xticks(rotation=rotation)
        plt.show()

        plt.figure(figsize=(width,height))

        plt.subplot(4,1,2)

        mean = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 21 EMA'].groupby('Extension')['Mean'].mean()).reset_index()
        count = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 21 EMA'].groupby('Extension')['Count'].sum()).reset_index()

        holder = pd.merge(count, mean, how='left', on='Extension')
        holder['Extension'] = holder['Extension'].astype(str)
        holder

        holder = holder[holder[holder['Count'] > 1].index[0]:holder[holder['Count'] > 1].index[-1]+1].reset_index(drop=True)

        # Creating the bar chart
        plt.bar(holder['Extension'], holder['Mean'])

        # Annotating the bars with 'Label' values
        for idx, row in holder.iterrows():
            plt.annotate(row['Count'], (row['Extension'], row['Mean']), textcoords="offset points", xytext=(0,20), ha='center')

        plt.xlabel('Extension from 21 EMA')
        plt.ylabel('Mean Return')
        plt.title('21 EMA')
        plt.xticks(rotation=rotation)
        plt.show()

        plt.figure(figsize=(width,height))

        plt.subplot(4,1,3)

        mean = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 50 SMA'].groupby('Extension')['Mean'].mean()).reset_index()
        count = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 50 SMA'].groupby('Extension')['Count'].sum()).reset_index()

        holder = pd.merge(count, mean, how='left', on='Extension')
        holder['Extension'] = holder['Extension'].astype(str)
        holder

        holder = holder[holder[holder['Count'] > 1].index[0]:holder[holder['Count'] > 1].index[-1]+1].reset_index(drop=True)

        # Creating the bar chart
        plt.bar(holder['Extension'], holder['Mean'])

        # Annotating the bars with 'Label' values
        for idx, row in holder.iterrows():
            plt.annotate(row['Count'], (row['Extension'], row['Mean']), textcoords="offset points", xytext=(0,20), ha='center')

        plt.xlabel('Extension from 50 SMA')
        plt.ylabel('Mean Return')
        plt.title('50 SMA')
        plt.xticks(rotation=rotation)
        plt.show()

        plt.figure(figsize=(width,height))

        plt.subplot(4,1,4)

        mean = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 200 SMA'].groupby('Extension')['Mean'].mean()).reset_index()
        count = pd.DataFrame(results[results['Moving Average'] == 'Open Distance 200 SMA'].groupby('Extension')['Count'].sum()).reset_index()

        holder = pd.merge(count, mean, how='left', on='Extension')
        holder['Extension'] = holder['Extension'].astype(str)
        holder

        holder = holder[holder[holder['Count'] > 1].index[0]:holder[holder['Count'] > 1].index[-1]+1].reset_index(drop=True)

        # Creating the bar chart
        plt.bar(holder['Extension'], holder['Mean'])

        # Annotating the bars with 'Label' values
        for idx, row in holder.iterrows():
            plt.annotate(row['Count'], (row['Extension'], row['Mean']), textcoords="offset points", xytext=(0,20), ha='center')

        plt.xlabel('Extension from 200 SMA')
        plt.ylabel('Mean Return')
        plt.title('200 SMA')
        plt.xticks(rotation=rotation)
        plt.show()

# Link the button to the function
button.on_click(on_button_clicked)

Text(value='SPY', description='Ticker:')

Text(value='2022-01-01', description='Start Date:')

IntText(value=5, description='Days:')

Button(description='Run Analysis', style=ButtonStyle())

Output()